In [2]:
# importation
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import os
import signal
import joblib

In [3]:
# load le dataset
X = joblib.load('X_test.pkl')
y = joblib.load('y_test.pkl')

In [4]:
with open('lr_model.pkl', 'rb') as file : 
    lr_model = pickle.load(file)
    
dt_model = joblib.load('dt_model.pkl')

best_rf_model = joblib.load('best_rf_model.pkl')

In [5]:
predictions_lr = lr_model.predict(X)

c:\Users\pphong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [6]:
predictions_dt = dt_model.predict(X)

c:\Users\pphong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


In [7]:
predictions_rf = best_rf_model.predict(X)

c:\Users\pphong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [9]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Create the layout of the app
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Solar Energy Prediction Dashboard"), className="mb-4")),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='model-dropdown',
                options=[
                    {'label': 'Linear Regression', 'value': 'Linear Regression'},
                    {'label': 'Decision Tree', 'value': 'Decision Tree'},
                    {'label': 'Random Forest', 'value': 'Random Forest'}
                ],
                value='Linear Regression',
                placeholder="Select a model"
            ),
            dbc.Button("Show Predictions", id="show-predictions", className="mt-2"),
            dbc.Button("Stop Dashboard", id="stop-dashboard", className="mt-2", color="danger")
        ], width=3),
        dbc.Col(dcc.Graph(id='prediction-graph'), width=9)
    ])
], fluid=True)

dataImputed2 = pd.DataFrame({
    'dtm': pd.date_range(start='2023-10-28', periods=500, freq='D'),
    'Solar_MW': np.random.rand(500) * 100
})

# Define callback to update graph
@app.callback(
    Output('prediction-graph', 'figure'),
    Input('show-predictions', 'n_clicks'),
    State('model-dropdown', 'value')
)
def update_graph(n_clicks, selected_model):
    if n_clicks is None:
        return go.Figure()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dataImputed2['dtm'], y=y, mode='lines+markers', name='Données réelles'))

    if selected_model == 'Linear Regression':
        fig.add_trace(go.Scatter(x=dataImputed2['dtm'], y=predictions_lr, mode='lines+markers', name='Prédictions'))
    elif selected_model == 'Decision Tree':
        fig.add_trace(go.Scatter(x=np.arange(len(y)), y=predictions_dt, mode='lines', name='Decision Tree'))
    elif selected_model == 'Random Forest':
        fig.add_trace(go.Scatter(x=np.arange(len(y)), y=predictions_rf, mode='lines', name='Random Forest'))

    fig.update_layout(title='Comparaison des Prédictions et des Données Réelles', xaxis_title='Date', yaxis_title='Solar MW')

    return fig

# Define callback to stop the server
@app.callback(
    Output('stop-dashboard', 'disabled'),
    Input('stop-dashboard', 'n_clicks')
)
def stop_server(n_clicks):
    if n_clicks is not None:
        os.kill(os.getpid(), signal.SIGTERM)
    return False

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # Utilisez un numéro de port différent si nécessaire

In our weather forecasting project, we evaluated three different regression models: Random Forest Regressor, Linear Regression, and Decision Tree Regressor. Despite expecting higher precision, our models demonstrated a satisfactory level of accuracy, with the predicted curves closely mirroring the actual trends.

We selected these models due to their established effectiveness in regression tasks. Comparing Linear Regression, a fundamental regression technique, with the more sophisticated Random Forest Regressor highlighted the differences in performance. The Random Forest Regressor, as anticipated, delivered superior precision, validating its advanced capabilities.

Overall, the use of these models provided valuable insights into their respective strengths and limitations in weather prediction. We are satisfied with the predictions we achieved and are committed to further improving them in the future.